In [1]:
import pandas as pd
data = pd.read_csv("clusterized_train.csv")

In [2]:
data[data["cluster_EM_25"] == 25].head(30)

,id,start_date,role,text,message_date,cluster_EM_25


In [3]:
clusters = {0: "code/translit",
            1: "Восхищения",
            2: "Числа",
            3: "Благодарности",
            4: "Приветствия",
            5: "Вопросы",
            6: "У кого-то что-то",
            7: "Прощания",
            8: "Эмоции",
            9: "Что-то сделано",
            10: "Понимание",
            11: "Мусор",
            12: "Прощания",
            13: "я что-то сделал",
            14: "Технические шоколадки",
            15: "я что-то сделал",
            16: "кого-то не будет",
            17: "Восхищения",
            18: "Пожелания",
            19: "Эмоции",
            20: "Эмоции",
            21: "Мусор",
            22: "Ответы",
            23: "Мусор",
            24: "Мусор"}

In [4]:
data["cluster_EM_25"] = data["cluster_EM_25"].replace(clusters)

In [5]:
data

,id,start_date,role,text,message_date,cluster_EM_25
0,307751.0,"2024-3-6, 10:02",user,"winningConditions = [\n [0, 1, 2],\n [3,...",2024-03-06 10:53:03,Мусор
1,307752.0,"2024-3-13, 09:51",user,https://diresnode.com/; 2024-03-13T10:12:22.638Z,2024-03-13 10:12:22,Мусор
2,321813.0,"2024-3-2, 12:56",user,пока; 2024-03-02T14:29:01.773Z,2024-03-02 14:29:01,Понимание
3,321813.0,"2024-3-2, 12:56",user,пока; 2024-03-02T14:29:07.513Z,2024-03-02 14:29:07,Понимание
4,321813.0,"2024-3-2, 12:56",user,досвидания; 2024-03-02T14:29:08.331Z,2024-03-02 14:29:08,Прощания
...,...,...,...,...,...,...
114390,437109.0,"2024-3-31, 14:00",user,"Не поверите, он все это время на зарядке(",2024-03-31 15:21:12,У кого-то что-то
114391,437109.0,"2024-3-31, 14:00",user,"Видимо, отложить. Спасибо!; 2024-03-31T15:22:2...",2024-03-31 15:22:22,Эмоции
114392,437224.0,"2024-3-30, 06:54",user,Здравствуйте!; 2024-03-30T07:27:52.900Z,2024-03-30 07:27:52,Приветствия
114393,437258.0,"2024-3-30, 06:55",user,Здравствуйте; 2024-03-30T07:02:37.652Z,2024-03-30 07:02:37,Приветствия


In [6]:
import pickle
import numpy as np
import re
import fasttext

In [7]:
em_reload = pickle.load(open("em_25_on_50pca_from_fasttext.pkl",'rb'))

In [8]:
data = pd.read_csv("train_GB_KachestvoPrepodovaniya.csv")
data = data[data['ID урока'].notna() * data['Дата старта урока'].notna() * data["Дата сообщения"].notna()]
arr_dicts = data.groupby(["ID урока", "Дата старта урока"]).agg(list).to_dict('records')
df_list = [pd.DataFrame(arr_dicts[i]) for i in range(len(arr_dicts))]
df_list[2351]["Дата сообщения"][5] = "2024-03-28 15:31:36"
start_date = data["Дата старта урока"]
start_secs = pd.DataFrame(pd.DataFrame(start_date.str.split().tolist())[1].str.split(":").tolist())[0].astype(np.int64) * 3600 + pd.DataFrame(pd.DataFrame(start_date.str.split().tolist())[1].str.split(":").tolist())[1].astype(np.int64) * 60
i = 0
start_list= []
# start_secs = pd.DataFrame(pd.DataFrame(start_date.str.split().tolist())[1].str.split(":").tolist())[0].astype(np.int64) * 3600 + pd.DataFrame(pd.DataFrame(start_date.str.split().tolist())[1].str.split(":").tolist())[1].astype(np.int64) * 60
for df in df_list:
    start_list.append(start_secs[i:i+len(df)])
    i += len(df)
for i in range(len(df_list)):
    df = df_list[i]
    df["secs"] = np.array(3600 * pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[0].astype(np.int64) + 60 * pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[1].astype(np.int64) + pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[2].astype(np.int64))
    df["start_secs"] = np.array(start_list[i])
    df["time_since_start"] = np.array(3600 * pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[0].astype(np.int64) + 60 * pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[1].astype(np.int64) + pd.DataFrame(pd.DataFrame(df["Дата сообщения"].str.split().tolist())[1].str.split(":").tolist())[2].astype(np.int64)) - np.array(start_list[i])


/tmp/ipykernel_77/3405244745.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_list[2351]["Дата сообщения"][5] = "2024-03-28 15:31:36"
/tmp/ipykernel_77/3405244745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [9]:
pca_reload = pickle.load(open("pca_fasttext_300_to_50.pkl",'rb'))
model_path = 'ft_native_300_ru_wiki_lenta_remstopwords.bin'
model = fasttext.load_model(model_path)
text_vectorized = df_list[2]["Текст сообщения"].apply(lambda x: re.sub(r'[0-9-TZ:.;\n=\[\],]', '', str(x))).apply(lambda x: model.get_sentence_vector(x.lower())).values
text_vectorized = np.array([text.tolist() for text in text_vectorized])
result_new = pca_reload.transform(text_vectorized)
predicted = model.predict(result_new)
# text_vectorize

AttributeError: 'numpy.ndarray' object has no attribute 'find'

In [11]:
result_new

array([[-0.1525006 ,  0.01141987, -0.18823588, ..., -0.06938757,
        -0.19954535, -0.02575504],
       [-0.1525006 ,  0.01141987, -0.18823588, ..., -0.06938757,
        -0.19954535, -0.02575504],
       [-0.08807585,  0.05782858,  0.07406303, ..., -0.01800919,
         0.04238376, -0.04413281],
       ...,
       [ 0.01901825, -0.17191829,  0.04948949, ...,  0.00216772,
         0.01476174, -0.067897  ],
       [ 0.07267515, -0.2125472 ,  0.13153149, ...,  0.05928704,
        -0.09217316, -0.02670563],
       [ 0.08447668, -0.13341186,  0.14819002, ...,  0.06705518,
         0.04463497, -0.05653422]])

In [12]:
!pwd

/opt/app/jupyter
